In [119]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

## Модельная задача

### Аналитическое решение

In [147]:
k = 1. + np.cos(0.5) * np.cos(0.5)
q = 1.
f = np.sin(0.5) * np.sin(0.5)
beta1 = 1.
eps1 = 0
beta2 = 1.
eps2 = 1.

print(np.sqrt(1. / (1 + np.cos(0.5) * np.cos(0.5))))
print(f/q)
lmb = lambda q,k: np.sqrt(q/k)
lmb1 = lmb(q,k)
lmb2 = lmb1
c1 = ((k*lmb2 + beta2)*(beta1*f - eps1*q) * np.exp(lmb2) + (k*lmb2 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb1 - beta1)*(k*lmb2 + beta2)*np.exp(lmb2) - (k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1)))

0.7516139358484979
0.22984884706593015


/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_57178/2390821884.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  c1 = ((k*lmb2 + beta2)*(beta1*f - eps1*q) * np.exp(lmb2) + (k*lmb2 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb1 - beta1)*(k*lmb2 + beta2)*np.exp(lmb2) - (k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1)))


In [152]:
def analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2):
    lmb = lambda q,k: np.sqrt(q/k)
    u = lambda c1,c2,lmb,x,f,q: c1 * np.exp(lmb*x) + c2 * np.exp(-lmb*x) + f/q
    lmb1 = lmb(q,k)
    lmb2 = - lmb1
    c1 = ((k*lmb2 + beta2)*(beta1*f - eps1*q) * np.exp(lmb2) + (k*lmb2 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb1 - beta1)*(k*lmb2 + beta2)*np.exp(lmb2) - (k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1)))
    print("c1=", c1)
    c2 = ((k*lmb1 + beta2)*(beta1*f - eps1*q) * np.exp(lmb1) + (k*lmb1 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1) - (k*lmb1 + beta2)*(k*lmb1 - beta1)*np.exp(lmb2)))
    print("c2=", c2)
    u1 = u(c1,c2,lmb1,x,f,q)
    return u1



In [153]:
def analytic_points(k,q,f,beta1,eps1,beta2,eps2):
        x1 = np.linspace(0,1,11)
        y1 = []
        for x in x1:
                res1 = analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2)
                y1.append(res1)
        return y1

### Метод прогонки

In [154]:
def progonkaMod(n,k,q,f,beta1,eps1,beta2,eps2):
    h = 1. / n
    x0 = 1
    solutions = np.zeros(11)
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = k
    bl = -2*k - q*h**2
    cl = k
    dl = -f*h**2
    a0 = k
    b0 = -k-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k-beta2*h
    cL=k
    dL = -eps2*h
    
    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al/(bl+cl*alpha[i-1])
        beta[i] = (dl - cl*beta[i-1])/(bl+cl*alpha[i-1])

    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [155]:
x = np.linspace(0,1,11)
u1 = progonkaMod(10,k,q,f,beta1,eps1,beta2,eps2)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
df = pd.DataFrame({"x": x, "u аналит.": a1, "u числ. ": u1 ,"delta u": delta1})
display(df)
print("delta u max =", max(delta1))

c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306
c1= 0.15342694478418903
c2= -0.07418751633228306


,x,u аналит.,u числ.,delta u
0,0.0,0.309088,0.311713,0.002625
1,0.1,0.326436,0.329323,0.002887
2,0.2,0.344330,0.347494,0.003164
3,0.3,0.362871,0.366330,0.003459
4,0.4,0.382163,0.385937,0.003774
5,0.5,0.402317,0.406426,0.004109
6,0.6,0.423445,0.427912,0.004467
7,0.7,0.445667,0.450517,0.004850
8,0.8,0.469110,0.474369,0.005260
9,0.9,0.493904,0.499602,0.005698


delta u max = 0.006168317423082548


## Задача с переменными коеффициентами

In [125]:
def progonkaVar(n,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11); x0 = 1; x = 0 + h
    solutions = np.zeros(11)
    k = lambda x: 1 + np.cos(x) * np.cos(x)
    q = lambda x: 1
    f = lambda x: np.sin(x) * np.sin(x)
    
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = lambda x: k(x+h/2)
    bl = lambda x: -(k(x+h/2)+k(x-h/2)+q(x)*h**2)
    cl = lambda x: k(x-h/2)
    dl = lambda x: -f(x)*h**2
    a0 = k(0)
    b0 = -k(0)-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k(1)-beta2*h
    cL = k(1)
    dL = -eps2*h

    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al(x)/(bl(x)+cl(x)*alpha[i-1])
        beta[i] = (dl(x) - cl(x)*beta[i-1])/(bl(x)+cl(x)*alpha[i-1])
        x += h
    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [126]:
x = np.linspace(0,1,11)
u2 = progonkaVar(1000,beta1,eps1,beta2,eps2)
a2 = progonkaVar(10**6,beta1,eps1,beta2,eps2)
delta2 = []
for i in range(len(u2)):
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u аналит.": a2, "u числ. ": u2 ,"delta u ": delta2})
display(df)
print("delta u max =", max(delta2))

,x,u аналит.,u числ.,delta u
0,0.0,-0.152739,-0.152953,0.000213
1,0.1,-0.160783,-0.161004,0.000221
2,0.2,-0.169780,-0.170009,0.000229
3,0.3,-0.180047,-0.180286,0.000239
4,0.4,-0.192053,-0.192304,0.000250
5,0.5,-0.206444,-0.206707,0.000263
6,0.6,-0.224073,-0.224351,0.000278
7,0.7,-0.246042,-0.246337,0.000296
8,0.8,-0.273735,-0.274052,0.000316
9,0.9,-0.308857,-0.309197,0.000340


delta u max = 0.000367894138417757


## Таблица результатов

In [127]:
x = np.linspace(0,1,11)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
a2 = progonkaVar(10**6,beta1,eps1,beta2,eps2)

In [128]:
delta1 = []
delta2 = []
u1 = progonkaMod(10,k,q,f,beta1,eps1,beta2,eps2)
u2 = progonkaVar(10,beta1,eps1,beta2,eps2)
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": u1 ,"du мод.": delta1, 
                  "u пер. аналит.": a2, "u пер. числ. ": u2 ,"du": delta2})
display(df)
print("du мод. макс =", max(delta1))
print("du перем. макс = ", max(delta2))

,x,u мод. аналит.,u мод. числ.,du мод.,u пер. аналит.,u пер. числ.,du
0,0.0,-0.153097,-0.173943,0.020846,-0.152739,-0.174273,0.021534
1,0.1,-0.163455,-0.183770,0.020315,-0.160783,-0.182987,0.022204
2,0.2,-0.176036,-0.195933,0.019897,-0.169780,-0.192763,0.022983
3,0.3,-0.190911,-0.210501,0.019591,-0.180047,-0.203934,0.023887
4,0.4,-0.208164,-0.227557,0.019394,-0.192053,-0.216986,0.024933
5,0.5,-0.227892,-0.247197,0.019305,-0.206444,-0.232591,0.026147
6,0.6,-0.250208,-0.269532,0.019324,-0.224073,-0.251632,0.027559
7,0.7,-0.275237,-0.294688,0.019452,-0.246042,-0.275250,0.029208
8,0.8,-0.303120,-0.322808,0.019687,-0.273735,-0.304877,0.031141
9,0.9,-0.334016,-0.354049,0.020033,-0.308857,-0.342271,0.033414


du мод. макс = 0.020846407892085944
du перем. макс =  0.03609480323700193
